Connected to Python 3.10.11

4.2 최적화 구현 단계의 step1~step5 까지의 4.3 서비스 활용 시나리오 출력 결과 

In [35]:

import mysql.connector # mysql workbench에 접근하기 위한 라이브러리
import pandas as pd # DB에서 가져온 테이블을 dataframe으로 바꾸기 위한 pandas 라이브러리

# mysql workbench에 접근하기 위해 필요한 workbench 정보 입력
host = '127.0.0.1'
user = 'root'
password = 'autoset'
database = 'busroute'

# MySQL server에 mysql.connector.connect를 이용해 연결한다. 
conn = mysql.connector.connect(host=host, user=user, password=password, database=database)

# SQL 쿼리를 실행할 커서 개체를 만든다. 
''' 커서 객체는 sql 쿼리를 실행하고 결과를 가져오는데 사용된다.'''
cursor = conn.cursor()

# sql 쿼리를 실행하여 테이블의 값들을 가져온다. 
table1_query = 'SELECT * FROM disabilityinformation'
cursor.execute(table1_query)
table1_data = cursor.fetchall()
columns1 = [i[0] for i in cursor.description]

'''sql 쿼리를 정의하고 실행한다.

그 결과를 fetchall() 메서드를 사용해 가져오고 
각 열의 이름을 'cursor.description'에서 추출하여
columns1에 저장한다. 
fetchall:쿼리 결과 집합의 모든 행을 검색하는 메소드 -> table1_data 변수에 저장
cursor.description: 열에 대한 정보 추출할 때 사용 -> colums1에 리스트로 저장'''

table2_query = 'SELECT * FROM passengerdisabilitycode'
cursor.execute(table2_query)
table2_data = cursor.fetchall()
columns2 = [i[0] for i in cursor.description]

table3_query = 'SELECT * FROM shuttlebus'
cursor.execute(table3_query)
table3_data = cursor.fetchall()
columns3 = [i[0] for i in cursor.description]


table4_query = 'SELECT * FROM destinationlocation'
cursor.execute(table4_query)
table4_data = cursor.fetchall()
columns4 = [i[0] for i in cursor.description]

#연결된 커서와 데이터베이스 연결을 닫는다.
cursor.close()
conn.close()

# 읽어온 각 테이블의 정보를 데이터 프레임으로 만들어 각각의 변수에 할당한다. 
table1_df = pd.DataFrame(table1_data, columns=columns1)
table2_df = pd.DataFrame(table2_data, columns=columns2)
table3_df = pd.DataFrame(table3_data, columns=columns3)
table4_df = pd.DataFrame(table4_data, columns=columns4)

# 만든 최종 데이터 프레임들을 출력한다. 
print('Table 1 DataFrame:')
print(table1_df)


print('\nTable 2 DataFrame:')
print(table2_df)

print('\nTable 3 DataFrame:')
print(table3_df)

print('\nTable 4 DataFrame:')
print(table4_df)

Table 1 DataFrame:
   disability_code disability_type severity
0            A00\r             자폐성       중증
1            A01\r             자폐셩       경증
2             AR00              청각       중증
3             AR01              청각       경증
4            B00\r             뇌병변       중증
5            B01\r             뇌병변       경증
6            E00\r             뇌전증       중증
7            E01\r             뇌전증       경증
8            F00\r              안면       중증
9            F01\r              안면       경증
10           H00\r              심장       중증
11           H01\r              심장       경증
12           I00\r              지적       중증
13           I01\r              지적       경증
14           K00\r              신장       중증
15           K01\r              신장       경증
16           L00\r               간       중증
17           L01\r               간       경증
18            LA00              언어       중증
19            LA01              언어       경증
20           M00\r              정신       중증
21           

In [36]:
disabilityinformation = table1_df 

passengerdisabilitycode = table2_df 

riding_history = table3_df

destination_location = table4_df 

In [37]:
#위도, 경도, 장애등록번호 컬럼을 Select
selected_columns = ['Passenger_Location_Latitude', 'Passenger_Location_Longitude', 'disabilityID']

#subset_data에 호출 목록 테이블에서 목적지 코드가 'W018'인 곳의 위도, 경도, 고객의 장애등록번호를 저장.
subset_data = riding_history.loc[riding_history['destination_code'] == 'E018', selected_columns]

# 동일한 목적지를 가는 동일한 고객의 기록이 두 개인 경우, 하나를 선택, 즉 중복을 제거
# 중복을 제거한 특정 목적지 'W018'에 가고자 하는 고객의 장애등록번호의 배열을 'info_list'로 지정  
# 이로써 info_list에는 중복되는 고객의 정보(장애등록번호)가 없음 
info_list = set(subset_data['disabilityID'])

In [38]:
# 한 사람이 여러 개의 장애 코드를 가질 수 있기에, 장애 코드 테이블에서 'WO18'을 가고자 하는
# 'info_list'에 해당하는 고객의 정보만 추출하여 result_disability_code 에 저장 
result_disability_code = passengerdisabilitycode[passengerdisabilitycode['disabilityID'].isin(info_list)]

# W018에 가고자 하는 고객의 정보만 모은 장애 코드 테이블에서 장애 코드 컬럼의 값만 list_of_code에 저장. 
list_of_code = list(result_disability_code['disability_code'])

In [39]:
mydir = r'C:\Users\yeeun\OneDrive\바탕 화면\3-2\엔지니어링DB\데이터 생성 csv\최종엑셀'

# 장애 코드 별 가중치에 대한 엑셀파일을 불러옴. 
disability_weight = pd.read_excel(mydir+'\\장애 코드별 가중치.xlsx') 

# Clean the data by stripping whitespaces and carriage returns
result_disability_code['disability_code'] = result_disability_code['disability_code'].str.strip().str.replace(r'\r', '')
disability_weight['장애코드'] = disability_weight['장애코드'].str.strip().str.replace(r'\r', '')

<ipython-input-39-3bf4b6f2ef5a>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_disability_code['disability_code'] = result_disability_code['disability_code'].str.strip().str.replace(r'\r', '')


In [40]:
# '가중치'라는 컬럼을 만들어서 특정 장애코드에 해당하는 가중치를 대입
result_disability_code['weight'] = disability_weight.set_index('장애코드')['가중치'].loc[result_disability_code['disability_code']].values

# 장애등록번호와 장애코드 두 개의 컬럼 값이 모두 동일하다면, 즉 완전히 중복되는 정보일 경우 제거. 
result_disability_code.drop_duplicates(subset=['disability_code', 'disabilityID'], keep='first', inplace=True)

# 장애가 여러 개 있는 고객도 있기에 가중치를 합하여 'sum of weight'이라는 컬럼 만들어줌. 
weighted_sum_df = result_disability_code.groupby('disabilityID')['weight'].sum().reset_index(name='sum of weight')

# 앞서 만든 subset_data라는 테이블(목적지 코드가 'W018'인 곳의 위도, 경도, 고객의 장애등록번호)과
# 고객별 장애 유형, 중증도에 따른 가중치 합 컬럼이 합쳐진 'merged_df'라는 데이터프레임을 만들어서 저장 
merged_df = pd.merge(subset_data, weighted_sum_df[['disabilityID', 'sum of weight']], left_on='disabilityID', right_on='disabilityID', how='left')

<ipython-input-40-28f3bc3a487c>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_disability_code['weight'] = disability_weight.set_index('장애코드')['가중치'].loc[result_disability_code['disability_code']].values
<ipython-input-40-28f3bc3a487c>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_disability_code.drop_duplicates(subset=['disability_code', 'disabilityID'], keep='first', inplace=True)


In [41]:
# 고객의 수가 10 이상인 경우, 해당 셀을 실행하여 가중치 값이 높은 특정 고객의 현위치만 정류소로 선정

# 가중치 오름차순으로 정렬 
merged_df_sorted = merged_df.sort_values(by='sum of weight', ascending = False)

# 전체 고객 수에서 가중치가 높은 75%의 고객의 현위치만 추출
percentage = 0.75

rows_to_keep = round(len(merged_df_sorted) * percentage)

#new_df 라는 새로운 데이터프레임으로 저장
#new_df는 ('장애등록번호', '위도', '경도', '가중치합')에 대한 정보가 저장된 데이터프레임 
new_df = merged_df_sorted.head(rows_to_keep)

In [42]:
# 목적지의 위치(위도, 경도)와 고객의 현위치(위도, 경도)를 합쳐 위치 정보로서 저장
selected = ['destination_latitude', 'destination_longitude']
des_x_y = destination_location.loc[destination_location['destination_code'] == 'E018', selected]

In [43]:
points_x = list(new_df['Passenger_Location_Longitude']) # 경도 

points_y = list(new_df['Passenger_Location_Latitude']) # 위도 

# points_x는 노드의 위도 정보를 담은 리스트, points_y는 노드의 경도 정보를 담은 리스트 
points_x.append(des_x_y.iloc[0,1])
points_y.append(des_x_y.iloc[0,0])
x_str = [str(value) for value in points_x]
y_str = [str(value) for value in points_y]

print(x_str)
print(y_str)

['126.8533266789011', '126.8009165760052', '126.8497874229351', '126.7867543796751', '126.7866127944142', '126.8103656606485', '126.789881276027', '126.8265537002391', '126.8636211']
['37.55857135261515', '37.56491768571411', '37.56556715776191', '37.57486868553623', '37.54867709015525', '37.57006611649995', '37.54641943268085', '37.562099718106', '37.56043066']


In [45]:
import requests

# Replace YOUR_REST_API_KEY with your actual Kakao API key
REST_API_KEY = "8f86dd860f90ae51e8c091ccad3fbe75"

url = "https://apis-navi.kakaomobility.com/v1/origins/directions"

headers = {
    "Content-Type": "application/json",
    "Authorization": f"KakaoAK {REST_API_KEY}"
}

points = [{'x': x, 'y': y, 'key': str(i)} for i, (x, y) in enumerate(zip(x_str, y_str))]

num_points = len(points)
distance_matrix = [[0] * num_points for _ in range(num_points)]

In [46]:
for i in range(num_points):
    for j in range(num_points):
        if i == j: 
            distance_matrix[i][j] = 0
        else:
            data = {
                'origins': [points[i]],
                'destination': points[j],
                'radius': 10000,
                'priority': 'DISTANCE'
            }

            response = requests.post(url, headers=headers, json=data)

            try:
                # Extract the distance information correctly
                distance = response.json()['routes'][0]['summary']['distance']

                distance_matrix[i][j] = distance
# 예은이 코드
# except (KeyError, IndexError, TypeError):
# print(f"Error processing response for points {i} to {j}. Check the API response structure.")

            except Exception as e:
                print(f"Error processing response for points {i} to {j}. Exception: {e}")
                print("Response content:", response.text)

# Print the distance matrix
for row in distance_matrix:
    print(row)

[0, 6276, 1144, 7317, 8888, 5023, 8497, 3435, 1291]
[6331, 0, 5658, 3904, 6799, 1995, 6408, 3429, 7145]
[1252, 5548, 0, 6589, 8674, 4295, 8283, 2891, 1611]
[7421, 3145, 6585, 0, 6598, 3236, 8677, 5036, 8072]
[9262, 7115, 8589, 5907, 0, 7469, 1287, 6360, 10076]
[5139, 1917, 4303, 2958, 7020, 0, 6629, 2267, 5790]
[8502, 6355, 7829, 6440, 480, 6709, 0, 5600, 9316]
[4222, 3328, 3549, 5227, 6208, 2136, 5817, 0, 5036]
[1428, 6992, 1748, 8105, 9462, 5811, 9071, 4089, 0]
